# LLM Council - Backend Demo

This notebook demonstrates how to run the LLM Council backend code directly without needing the FastAPI server or React frontend.

## What is LLM Council?

LLM Council runs a 3-stage process:
1. **Stage 1**: Multiple LLMs provide individual responses to your question
2. **Stage 2**: Each LLM ranks the other responses (anonymized)
3. **Stage 3**: A "Chairman" LLM synthesizes all responses into a final answer

---

## Step 1: Install Required Dependencies

Run this cell to install all necessary packages:

In [ ]:
# Install dependencies
!pip install fastapi uvicorn python-dotenv httpx pydantic

## Step 2: Set Up Environment Variables

**IMPORTANT**: You need a Groq API key to use this notebook.

1. Get your free API key from: https://console.groq.com/keys
2. Replace `"your_groq_api_key_here"` below with your actual API key

In [ ]:
import os

# Set your Groq API key here
os.environ["GROQ_API_KEY"] = "your_groq_api_key_here"

# Verify it's set
if os.environ["GROQ_API_KEY"] == "your_groq_api_key_here":
    print("⚠️  WARNING: Please replace 'your_groq_api_key_here' with your actual Groq API key!")
else:
    print("✓ API key is set")

## Step 3: Import Backend Modules

Load all the LLM Council backend code:

In [ ]:
# Import backend modules
import sys
sys.path.insert(0, os.path.abspath('.'))

from backend.council import (
    stage1_collect_responses,
    stage2_collect_rankings,
    stage3_synthesize_final,
    calculate_aggregate_rankings,
    run_full_council
)
from backend.config import COUNCIL_MODELS, CHAIRMAN_MODEL

print("✓ Backend modules imported successfully")
print(f"\nCouncil Members: {COUNCIL_MODELS}")
print(f"Chairman Model: {CHAIRMAN_MODEL}")

## Step 4: Configure Your Question

Enter the question you want to ask the LLM Council:

In [ ]:
# Your question for the council
user_query = "What are the key differences between supervised and unsupervised machine learning?"

print(f"Question: {user_query}")

## Step 5: Run the Complete Council Process

This will run all 3 stages automatically:

In [ ]:
import asyncio

# Run the full council process
print("🏛️  Starting LLM Council...\n")

stage1_results, stage2_results, stage3_result, metadata = await run_full_council(user_query)

print("✓ Council process complete!")

## Step 6: View Stage 1 Results (Individual Responses)

See what each LLM said individually:

In [ ]:
print("=" * 80)
print("STAGE 1: INDIVIDUAL RESPONSES")
print("=" * 80)

for i, result in enumerate(stage1_results, 1):
    print(f"\n[{i}] Model: {result['model']}")
    print("-" * 80)
    print(result['response'])
    print()

## Step 7: View Stage 2 Results (Peer Rankings)

See how each LLM ranked the others:

In [ ]:
print("=" * 80)
print("STAGE 2: PEER RANKINGS")
print("=" * 80)

# Show the anonymous label mapping
print("\n📋 Anonymous Response Labels:")
for label, model in metadata['label_to_model'].items():
    print(f"  {label} = {model}")

# Show each model's ranking
for i, result in enumerate(stage2_results, 1):
    print(f"\n[{i}] Ranking by: {result['model']}")
    print("-" * 80)
    print(result['ranking'])
    print()

## Step 8: View Aggregate Rankings

See the overall consensus on which responses were best:

In [ ]:
print("=" * 80)
print("AGGREGATE RANKINGS (Lower average rank = Better)")
print("=" * 80)

for i, ranking in enumerate(metadata['aggregate_rankings'], 1):
    print(f"{i}. {ranking['model']}")
    print(f"   Average Rank: {ranking['average_rank']}")
    print(f"   Times Ranked: {ranking['rankings_count']}\n")

## Step 9: View Stage 3 Result (Final Synthesis)

See the Chairman's final answer:

In [ ]:
print("=" * 80)
print("STAGE 3: FINAL SYNTHESIS BY CHAIRMAN")
print("=" * 80)

print(f"\nChairman Model: {stage3_result['model']}")
print("-" * 80)
print(stage3_result['response'])
print()

---

## Alternative: Run Stages Individually

If you want more control, you can run each stage separately:

### Stage 1: Collect Individual Responses

In [ ]:
# Run Stage 1 only
custom_query = "Explain quantum computing in simple terms."

print(f"Question: {custom_query}\n")
stage1 = await stage1_collect_responses(custom_query)

for i, result in enumerate(stage1, 1):
    print(f"\n[{i}] {result['model']}:")
    print("-" * 60)
    print(result['response'][:200] + "...")  # Show first 200 chars

### Stage 2: Collect Rankings

In [ ]:
# Run Stage 2 on the Stage 1 results
stage2, label_map = await stage2_collect_rankings(custom_query, stage1)

print("Label Mapping:")
for label, model in label_map.items():
    print(f"  {label} = {model}")

print("\nParsed Rankings:")
for result in stage2:
    print(f"  {result['model']}: {result['parsed_ranking']}")

### Stage 3: Synthesize Final Answer

In [ ]:
# Run Stage 3 to get final synthesis
stage3 = await stage3_synthesize_final(custom_query, stage1, stage2)

print(f"Chairman ({stage3['model']}) says:\n")
print(stage3['response'])

---

## Bonus: Test with Different Questions

Try asking different types of questions:

In [ ]:
# Example questions you can try:
example_questions = [
    "What is the meaning of life?",
    "How do neural networks learn?",
    "What are the pros and cons of remote work?",
    "Explain blockchain technology to a 10-year-old.",
    "What makes a good software engineer?"
]

# Pick one and run it through the council
test_query = example_questions[1]  # Change the index to try different questions

print(f"Testing with: {test_query}\n")
results = await run_full_council(test_query)

# Show only the final answer
print("=" * 80)
print("FINAL ANSWER:")
print("=" * 80)
print(results[2]['response'])  # results[2] is stage3_result

---

## Summary

This notebook demonstrates the LLM Council backend functionality:

1. ✅ Stage 1: Collect responses from multiple LLMs
2. ✅ Stage 2: Have each LLM rank the responses
3. ✅ Stage 3: Synthesize final answer with a Chairman LLM

**Note**: The full web application (with FastAPI backend + React frontend) provides a better user experience with:
- Nice UI for viewing responses in tabs
- Conversation history
- Real-time streaming updates

To run the full application, use the `./start.sh` script as described in the README.
